In [4]:
!pip install -q transformers accelerate bitsandbytes peft qwen-vl-utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.6 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration

model_id = "isaacderhy/tyerce-qwen2.5-vl-lora"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

print("✅ Modèle Tyerce rechargé")


config.json: 0.00B [00:00, ?B/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

In [ ]:
# 📥 Imports
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor

# ⚙️ ID de ton repo Hugging Face
model_id = "isaacderhy/tyerce-qwen2.5-vl-lora"

# 🧠 Charger le modèle fine-tuné
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",     # auto place sur GPU si dispo
    torch_dtype="auto"     # adapte selon le device
)

# 🔧 Charger le processor (tokenizer + vision)
processor = AutoProcessor.from_pretrained(model_id)

print("✅ Modèle + processor chargés depuis Hugging Face :", model_id)


In [ ]:
import torch
import json
import glob
from lxml import etree
from PIL import Image
from qwen_vl_utils import process_vision_info

# ============================================
# 🔎 Chargement du RAG (lois en XML)
# ============================================
rag_file = "Tyerce-data-legalV2XML.xml"  # 👉 fichier XML contenant les articles
tree = etree.parse(rag_file)
root = tree.getroot()

def rag_search(query, max_articles=5):
    """Recherche simple dans le RAG XML et renvoie les articles pertinents."""
    results = []
    for article in root.findall(".//article"):
        texte = article.text or ""
        if query.lower() in texte.lower():
            results.append(texte.strip())
        if len(results) >= max_articles:
            break
    return results


# ============================================
# 🔎 Chargement d’une discussion JSON
# ============================================
def charger_discussion_json():
    """
    Cherche automatiquement le premier fichier JSON dans le dossier courant
    et renvoie le texte concaténé de la discussion + le nom du fichier.
    Format attendu :
    [
      {"auteur": "Acheteur", "message": "J'ai payé mais rien reçu"},
      {"auteur": "Vendeur", "message": "Je vais envoyer bientôt"}
    ]
    """
    fichiers = glob.glob("*.json")
    if not fichiers:
        raise FileNotFoundError("Aucun fichier JSON trouvé dans le dossier.")

    fichier = fichiers[0]  # on prend le premier trouvé
    with open(fichier, "r", encoding="utf-8") as f:
        discussion = json.load(f)

    # Construit le texte du litige à partir des messages
    litige_text = "\n".join([f"{m['auteur']}: {m['message']}" for m in discussion])
    return litige_text, fichier


# ============================================
# 🔎 Construction du prompt
# ============================================
def build_messages(litige_text, query, image_paths=None):
    rag_context = "\n".join(rag_search(query, max_articles=5))

    base_text = f"""
Tu es une IA juge de médiation pour Tyerce (tiers de confiance entre particuliers).
Analyse la discussion suivante et les preuves pour déterminer le litige,
puis rends un jugement basé uniquement sur la loi française et les extraits du RAG.
Tu dois faire des recherches su

### DISCUSSION ENTRE LES PARTIES
{litige_text}

### LOIS PERTINENTES DU RAG
{rag_context if rag_context else "Aucun article trouvé."}

### INSTRUCTION
Ton jugement sera fait d'une phrase courte, qui dans un premier temps résume le litige, dans un second temps apporte les articles de loi utilisés
puis finalement, la décision claire, en utilisant des termes comme, Le payeur (pour parler de celui qui a fait le payement) et le receveur (pour celui sensé recevoir le payement)


Ne donne aucune explication supplémentaire lorsque ce n'est pas nécessaire.
"""

    content = []
    if image_paths:
        for path in image_paths:
            try:
                img = Image.open(path).convert("RGB").resize((448, 448))
                content.append({"type": "image", "image": img})
            except Exception as e:
                print(f"⚠️ Impossible de charger l'image {path}: {e}")

    content.append({"type": "text", "text": base_text})
    return [{"role": "user", "content": content}]


# ============================================
# 📝 Exemple d’utilisation
# ============================================
import glob

# ============================================
litige_text, fichier_utilise = charger_discussion_json()
print(f"✅ Discussion chargée depuis {fichier_utilise}")

# 👉 Charge toutes les images du dossier courant (jpg, png, jpeg)
image_paths = glob.glob("*.jpg") + glob.glob("*.jpeg") + glob.glob("*.png")

if not image_paths:
    print("⚠️ Aucune image trouvée dans le dossier.")
else:
    print(f"✅ {len(image_paths)} image(s) trouvée(s): {image_paths}")

messages = build_messages(litige_text, query="livraison", image_paths=image_paths)


# ============================================
# 🤖 Génération avec Qwen-VL
# ============================================
input_text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[input_text],
    images=image_inputs,
    return_tensors="pt",
    padding=True
).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,   # plus déterministe
        temperature=0.3    # sortie concise et claire
    )

response = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print("=== RÉPONSE DU MODÈLE ===\n")
print(response)


In [ ]:
# ✅ Install propre : Streamlit + Transformers + dépendances, sans conflits
%pip -q install --upgrade pip

# Streamlit 1.37.1 exige Pillow < 11 → on le fixe
%pip -q install "streamlit==1.37.1" "pillow<11,>=7.1.0"

# Libs modèle + RAG + vision utilitaire Qwen
%pip -q install "transformers==4.56.0" "accelerate>=1.10.0" "lxml==5.4.0" "qwen-vl-utils==0.0.8" "torchvision"

# Désactive explicitement flash-attn/xformers (source des erreurs triton.ops)
!pip -q uninstall -y flash-attn flash_attn xformers || true

# — Vérification versions —
import streamlit, transformers, torch, PIL, lxml, torchvision
print("✅ Versions OK:")
print(" - streamlit", streamlit.__version__)
print(" - transformers", transformers.__version__)
print(" - torch", torch.__version__)
print(" - pillow", PIL.__version__)
print(" - lxml", lxml.__version__)


In [ ]:
%%writefile app.py
import os, io, json, textwrap, datetime
import streamlit as st
from lxml import etree
from PIL import Image
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
from qwen_vl_utils import process_vision_info

# Configuration de base
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["STREAMLIT_SERVER_FILE_WATCHER_TYPE"] = "none"

# Configuration de la page
st.set_page_config(
    page_title="Tyerce — Résolution de litige",
    page_icon="⚖️",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# Style CSS complet
st.markdown("""
<style>
@import url('https://fonts.googleapis.com/css2?family=Playfair+Display:wght@600;700&family=Inter:wght@400;500;600;700&family=JetBrains+Mono:wght@400;500&display=swap');

:root {
    --primary: #2D5A5A;
    --primary-dark: #1E3D3D;
    --accent: #FF6B6B;
    --accent-light: #FF9A8B;
    --neutral-50: #F8F9FA;
    --neutral-100: #F1F3F5;
    --neutral-200: #E9ECEF;
    --neutral-300: #DEE2E6;
    --neutral-400: #CED4DA;
    --neutral-500: #ADB5BD;
    --neutral-600: #6C757D;
    --neutral-700: #495057;
    --neutral-800: #343A40;
    --neutral-900: #212529;
    --glass-bg: rgba(255, 255, 255, 0.95);
    --glass-border: rgba(255, 255, 255, 0.3);
    --shadow-sm: 0 1px 2px rgba(0, 0, 0, 0.05);
    --shadow-md: 0 4px 6px rgba(0, 0, 0, 0.1);
    --shadow-lg: 0 10px 15px rgba(0, 0, 0, 0.1);
}

html, body, [data-testid="stAppViewContainer"] {
    min-height: 100vh;
    font-family: 'Inter', sans-serif;
    color: var(--neutral-900);
    background: linear-gradient(135deg, #F8F9FA 0%, #E9ECEF 100%);
}

.stApp > header { display: none; }
.block-container { max-width: 1400px; padding: 24px; }

/* Header */
.tyerce-header {
    text-align: center;
    margin: 20px 0 40px;
    padding: 16px 0;
}

.tyerce-logo {
    font-family: 'Playfair Display', serif;
    font-size: 48px;
    font-weight: 700;
    color: var(--primary);
    margin-bottom: 8px;
    letter-spacing: -1px;
}

.tyerce-subtitle {
    font-size: 16px;
    color: var(--neutral-600);
    font-weight: 500;
    letter-spacing: 1px;
    text-transform: uppercase;
}

/* Layout */
.main-layout {
    display: grid;
    grid-template-columns: 380px 1fr 380px;
    gap: 24px;
    margin-bottom: 24px;
}

@media (max-width: 1200px) {
    .main-layout { grid-template-columns: 1fr; gap: 24px; }
}

/* Cartes */
.premium-card {
    background: rgba(255, 255, 255, 0.3);
    backdrop-filter: blur(4px);
    border: none;
    border-radius: 0;
    padding: 16px;
    margin: 0;
    height: auto;
    min-height: 100%;
    box-shadow: none;
    border-left: 3px solid var(--primary);
}

/* Sections */
.section-header {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-bottom: 20px;
    padding-bottom: 12px;
    border-bottom: 1px solid var(--neutral-200);
}

.section-title {
    font-family: 'Playfair Display', serif;
    font-size: 20px;
    font-weight: 600;
    color: var(--neutral-900);
    margin: 0;
    position: relative;
    display: inline-block;
}

.section-title::after {
    content: '';
    position: absolute;
    bottom: -6px;
    left: 0;
    width: 50%;
    height: 2px;
    background: linear-gradient(90deg, var(--primary), var(--accent));
}

.section-icon {
    width: 18px;
    height: 18px;
    color: var(--primary);
}

/* Statuts */
.status-grid {
    display: flex;
    flex-direction: column;
    gap: 8px;
    margin: 16px 0;
}

.status-item {
    display: flex;
    align-items: center;
    justify-content: space-between;
    padding: 8px 12px;
    border-radius: 8px;
    background: var(--neutral-50);
    border: 1px solid var(--neutral-200);
}

.status-label {
    font-size: 14px;
    font-weight: 500;
    color: var(--neutral-700);
}

.status-indicator {
    display: flex;
    align-items: center;
    gap: 8px;
    font-weight: 500;
}

.status-dot {
    width: 8px;
    height: 8px;
    border-radius: 50%;
}

.dot-success { background: #22C55E; }
.dot-warning { background: #F59E0B; }

/* Téléphone */
.phone-container {
    margin: 20px auto;
    max-width: 320px;
}



.phone-notch {
    position: absolute;
    top: 8px;
    left: 50%;
    transform: translateX(-50%);
    width: 120px;
    height: 24px;
    background: #000;
    border-radius: 16px;
}

.phone-header {
    position: absolute;
    top: 40px;
    left: 16px;
    right: 16px;
    display: flex;
    justify-content: space-between;
    align-items: center;
    color: #EEE;
    font-size: 12px;
}

.sms-container {
    position: absolute;
    top: 80px;
    bottom: 20px;
    left: 0;
    right: 0;
    padding: 16px;
    overflow-y: auto;
}

.sms-message {
    margin-bottom: 12px;
    max-width: 80%;
    animation: fadeIn 0.3s ease-out;
}

.sms-bubble {
    padding: 10px 14px;
    border-radius: 18px;
    font-size: 14px;
    line-height: 1.4;
    word-wrap: break-word;
    box-shadow: 0 1px 2px rgba(0,0,0,0.1);
}

.sms-left {
    background: #E5E7EB;
    color: #1F2937;
    border-bottom-left-radius: 4px;
    margin-right: auto;
}

.sms-right {
    background: var(--primary);
    color: white;
    border-bottom-right-radius: 4px;
    margin-left: auto;
}

.sms-author {
    font-size: 11px;
    font-weight: 600;
    margin-bottom: 4px;
    opacity: 0.8;
}

.sms-time {
    font-size: 10px;
    text-align: right;
    opacity: 0.7;
    margin-top: 4px;
}

.sms-empty {
    text-align: center;
    color: #9CA3AF;
    font-style: italic;
    margin: 60px 0;
}

/* Galerie */
.gallery {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(100px, 1fr));
    gap: 12px;
    margin-top: 16px;
}

.gallery img {
    width: 100%;
    aspect-ratio: 1;
    object-fit: cover;
    border-radius: 8px;
    border: 1px solid var(--neutral-200);
}

/* Verdict */
.verdict-container {
    background: var(--neutral-50);
    border: 1px solid var(--neutral-200);
    border-radius: 12px;
    padding: 20px;
    margin-top: 20px;
}

.verdict-header {
    display: flex;
    align-items: center;
    gap: 12px;
    margin-bottom: 16px;
}

.verdict-text {
    font-size: 16px;
    line-height: 1.6;
    color: var(--neutral-800);
    font-weight: 500;
}

/* Inputs */
.stTextInput input, .stTextArea textarea {
    background: var(--neutral-50) !important;
    border: 1px solid var(--neutral-300) !important;
    border-radius: 8px !important;
    padding: 10px 14px !important;
    font-size: 14px !important;
}

.stTextInput input:focus, .stTextArea textarea:focus {
    border-color: var(--primary) !important;
    box-shadow: 0 0 0 2px rgba(45, 90, 90, 0.1) !important;
}

/* Uploaders */
.upload-grid {
    display: grid;
    grid-template-columns: 1fr;
    gap: 16px;
    margin-bottom: 20px;
}

[data-testid="stFileUploader"] {
    background: var(--neutral-50) !important;
    border: 2px dashed var(--neutral-300) !important;
    border-radius: 8px !important;
    padding: 16px !important;
}

[data-testid="stFileUploader"]:hover {
    border-color: var(--primary) !important;
    background: rgba(45, 90, 90, 0.05) !important;
}

/* Boutons */
.stButton button {
    border-radius: 8px !important;
    padding: 10px 16px !important;
    font-weight: 500 !important;
    border: 1px solid var(--neutral-300) !important;
    background: var(--neutral-50) !important;
    color: var(--neutral-800) !important;
    transition: all 0.2s ease !important;
}

.stButton button:hover {
    transform: translateY(-1px) !important;
    box-shadow: var(--shadow-sm) !important;
}

.stButton button[kind="primary"] {
    background: linear-gradient(135deg, var(--primary), var(--primary-dark)) !important;
    color: white !important;
    border: none !important;
    box-shadow: var(--shadow-sm) !important;
    font-weight: 600 !important;
}

/* Animation */
@keyframes fadeIn {
    from { opacity: 0; transform: translateY(10px); }
    to { opacity: 1; transform: translateY(0); }
}
</style>
""", unsafe_allow_html=True)

# Toutes les fonctions nécessaires
@st.cache_resource(show_spinner=False)
def _load_model_gpu(repo_id: str):
    processor = AutoProcessor.from_pretrained(repo_id)
    if hasattr(processor, "tokenizer") and processor.tokenizer.pad_token_id is None:
        processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id
    dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        repo_id,
        device_map="auto",
        torch_dtype=dtype,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
    )
    model.eval()
    return model, processor

@st.cache_resource(show_spinner=False)
def _load_model_cpu(repo_id: str):
    processor = AutoProcessor.from_pretrained(repo_id)
    if hasattr(processor, "tokenizer") and processor.tokenizer.pad_token_id is None:
        processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        repo_id,
        device_map={"": "cpu"},
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
    )
    model.eval()
    return model, processor

def _device_chip(model):
    if model is None:
        return "Non chargé", "warning"
    try:
        dev = next(model.parameters()).device
        style = "success" if "cuda" in str(dev) else "warning"
        return f"{str(dev).upper()}", style
    except:
        return "Non disponible", "warning"

def rag_search(tree: etree._ElementTree, query: str, max_articles=5):
    if tree is None or not query.strip(): return []
    root = tree.getroot()
    results = []
    for article in root.findall(".//article"):
        txt = (article.text or "").strip()
        if query.lower() in txt.lower():
            results.append(txt)
            if len(results) >= max_articles: break
    return results

def _load_rag_tree_from_upload(upload, fallback_text):
    if upload is not None:
        data = upload.read()
        try:
            return etree.parse(io.BytesIO(data))
        except Exception:
            root = etree.Element("rag")
            node = etree.SubElement(root, "article")
            node.text = data.decode("utf-8", errors="ignore")
            return etree.ElementTree(root)
    elif fallback_text.strip():
        root = etree.Element("rag")
        for a in [x.strip() for x in fallback_text.split("\n") if x.strip()]:
            node = etree.SubElement(root, "article")
            node.text = a
        return etree.ElementTree(root)
    return None

def _maybe_shrink_images(images):
    shrunk = []
    for img in images[:MAX_IMAGES]:
        try:
            cp = img.copy()
            cp.thumbnail((448,448))
            shrunk.append(cp.convert("RGB"))
        except: pass
    return shrunk

def build_messages(litige_text: str, rag_snippets, images):
    base_text = f"""
Tu es un juge expert en médiation pour Tyerce. Analyse la discussion suivante et les preuves visuelles fournies.
Utilise les articles de loi pertinents pour rendre un verdict clair, précis et professionnel en une seule phrase.
Corrige toute erreur d'orthographe ou de syntaxe dans ta réponse.

### DISCUSSION
{litige_text}

### ARTICLES DE LOI PERTINENTS
{os.linesep.join(rag_snippets) if rag_snippets else "Aucun article de loi pertinent trouvé."}

### FORMAT DE RÉPONSE
"En application des articles [numéros des articles] du [Code concerné], et au vu des éléments fournis,
le tribunal décide que [décision claire et précise avec sujet, verbe et complément]."
"""
    content = []
    for img in images:
        content.append({"type": "image", "image": img})
    content.append({"type": "text", "text": textwrap.dedent(base_text).strip()})
    return [{"role": "user", "content": content}]

def _prepare_inputs(processor, model, msgs):
    input_text = processor.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    image_inputs, _ = process_vision_info(msgs)
    inputs = processor(
        text=[input_text], images=image_inputs,
        return_tensors="pt", padding=True, truncation=True, max_length=MAX_INPUT_TOKENS,
    )
    dev = next(model.parameters()).device
    for k, v in list(inputs.items()):
        if hasattr(v, "to"): inputs[k] = v.to(dev)
    return inputs

# Constantes et état
MAX_IMAGES = 3
MAX_INPUT_TOKENS = 3072
MAX_NEW_TOKENS = 160
if "messages" not in st.session_state: st.session_state.messages = []
if "uploaded_images" not in st.session_state: st.session_state.uploaded_images = []
if "rag_tree" not in st.session_state: st.session_state.rag_tree = None
if "model" not in st.session_state:
    st.session_state.model = None
    st.session_state.processor = None
    st.session_state.model_id_loaded = None

# Header
st.markdown("""
<div class="tyerce-header">
  <div class="tyerce-logo">Tyerce</div>
  <div class="tyerce-subtitle">Résolution de Litiges</div>
</div>
""", unsafe_allow_html=True)

# Layout principal
st.markdown('<div class="main-layout">', unsafe_allow_html=True)

# COLONNE GAUCHE
st.markdown('<div class="premium-card">', unsafe_allow_html=True)
st.markdown('''
<div class="section-header">
  <svg class="section-icon" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="1.5">
    <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"/>
    <path d="M14 2v6h6"/>
    <path d="M16 13H8"/>
    <path d="M16 17H8"/>
    <path d="M10 9H8"/>
  </svg>
  <h2 class="section-title">Configuration Juridique</h2>
</div>
''', unsafe_allow_html=True)

# RAG d'abord
rag_xml = st.file_uploader("Base de données RAG (XML)", type=["xml"], help="Fichier XML contenant les articles de loi")
rag_fallback = st.text_area("Articles de loi", value="", height=100, placeholder="Collez ici les articles de loi pertinents...")

if st.button("Importer la Base RAG", use_container_width=True):
    st.session_state.rag_tree = _load_rag_tree_from_upload(rag_xml, rag_fallback)
    if st.session_state.rag_tree is not None:
        st.success("Base RAG importée ✅")
    else:
        st.warning("Aucune base RAG valide fournie")

# Puis modèle
model_id = st.text_input("Modèle Hugging Face", value="isaacderhy/tyerce-qwen2.5-vl-lora", placeholder="ex: Qwen/Qwen2.5-VL-7B")
force_cpu = st.toggle("Forcer CPU", value=False)

if st.button("Charger le Modèle", use_container_width=True):
    with st.spinner("Chargement..."):
        try:
            if force_cpu or not torch.cuda.is_available():
                model, processor = _load_model_cpu(model_id)
            else:
                model, processor = _load_model_gpu(model_id)
            st.session_state.model = model
            st.session_state.processor = processor
            st.success("Modèle chargé ✅")
        except Exception as e:
            st.error(f"Échec: {str(e)}")

# Statuts
mdl = st.session_state.model
dev_label, dev_style = _device_chip(mdl)
st.markdown('<div class="status-grid">', unsafe_allow_html=True)
model_status = "success" if mdl else "warning"
st.markdown(f'''
<div class="status-item">
  <span class="status-label">Modèle</span>
  <div>
    <span class="status-dot dot-{model_status}"></span>
    {"Chargé" if mdl else "Non chargé"}
  </div>
</div>
''', unsafe_allow_html=True)
rag_status = "success" if st.session_state.rag_tree is not None else "warning"
st.markdown(f'''
<div class="status-item">
  <span class="status-label">Base RAG</span>
  <div>
    <span class="status-dot dot-{rag_status}"></span>
    {"Disponible" if st.session_state.rag_tree else "Manquante"}
  </div>
</div>
''', unsafe_allow_html=True)
st.markdown(f'''
<div class="status-item">
  <span class="status-label">Device</span>
  <div>
    <span class="status-dot dot-{dev_style}"></span>
    {dev_label}
  </div>
</div>
''', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# COLONNE CENTRE
st.markdown('<div class="premium-card">', unsafe_allow_html=True)
st.markdown('''
<div class="section-header">
  <svg class="section-icon" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="1.5">
    <path d="M21 15a2 2 0 0 1-2 2H7l-4 4V5a2 2 0 0 1 2-2h14a2 2 0 0 1 2 2z"/>
  </svg>
  <h2 class="section-title">Discussion et Preuves</h2>
</div>
''', unsafe_allow_html=True)

# Uploads
up_disc = st.file_uploader("Discussion (JSON)", type=["json"], help="Format: [{'auteur': 'Acheteur', 'message': '...'}]")
if up_disc is not None:
    try:
        data = json.load(up_disc)
        if isinstance(data, list) and all(isinstance(x, dict) for x in data):
            st.session_state.messages = []
            for m in data:
                a, msg = m.get("auteur","Partie"), m.get("message","")
                if msg: st.session_state.messages.append({"auteur": a, "message": msg})
            st.success(f"{len(st.session_state.messages)} messages importés ✅")
        else:
            st.error("Format JSON invalide")
    except Exception as e:
        st.error(f"Erreur: {e}")

up_imgs = st.file_uploader("Preuves visuelles/Contrat", type=["png","jpg","jpeg","webp"], accept_multiple_files=True, help="Images, captures d'écran, contrat...")
if up_imgs:
    st.session_state.uploaded_images = []
    for f in up_imgs:
        try:
            im = Image.open(f).convert("RGB")
            st.session_state.uploaded_images.append(im)
        except Exception as e:
            st.warning(f"Image ignorée: {e}")

# Actions
col1, col2 = st.columns(2)
with col1:
    if st.button("Effacer tout", use_container_width=True):
        st.session_state.messages = []
        st.session_state.uploaded_images = []
        st.rerun()
with col2:
    if st.button("Exemple", use_container_width=True):
        st.session_state.messages = [
            {"auteur":"Acheteur","message":"Bonjour, j'ai commandé un produit le 10/05 mais ne l'ai toujours pas reçu."},
            {"auteur":"Vendeur","message":"Votre colis a été expédié le 12/05 avec le suivi COL123456789FR."},
            {"auteur":"Acheteur","message":"Le suivi indique 'problème de livraison' depuis 5 jours. Je demande un remboursement."}
        ]
        st.success("Exemple chargé ✅")

# Téléphone
st.markdown('<div class="phone-container">', unsafe_allow_html=True)

st.markdown('<div class="phone-notch"></div>', unsafe_allow_html=True)
st.markdown('<div class="phone-header">', unsafe_allow_html=True)
st.markdown(f'<div style="font-family: monospace; font-size: 12px;">{datetime.datetime.now().strftime("%H:%M")}</div>', unsafe_allow_html=True)
st.markdown('''
<div style="display: flex; gap: 8px; align-items: center;">
  <div style="width: 16px; height: 8px; border: 1px solid #EEE; border-radius: 2px;">
    <div style="width: 70%; height: 100%; background: #4CAF50; border-radius: 1px;"></div>
  </div>
  <div style="font-family: 'JetBrains Mono'; font-size: 12px;">4G</div>
  <div style="font-family: 'JetBrains Mono'; font-size: 12px;">📶</div>
</div>
''', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)
st.markdown('<div class="sms-container">', unsafe_allow_html=True)

if not st.session_state.messages:
    st.markdown('<div class="sms-empty">Aucune discussion en cours</div>', unsafe_allow_html=True)
else:
    for msg in st.session_state.messages:
        side = "right" if msg.get("auteur") == "Acheteur" else "left"
        st.markdown(f'''
        <div class="sms-message">
          <div class="sms-bubble sms-{side}">
            <div class="sms-author">{msg.get("auteur", "Partie")}</div>
            <div>{msg.get("message", "").strip()}</div>
            <div class="sms-time">{datetime.datetime.now().strftime("%H:%M")}</div>
          </div>
        </div>
        ''', unsafe_allow_html=True)

st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# Galerie
if st.session_state.uploaded_images:
    st.markdown('<div class="gallery">', unsafe_allow_html=True)
    cols = st.columns(3)
    for i, img in enumerate(_maybe_shrink_images(st.session_state.uploaded_images)):
        with cols[i % 3]:
            st.image(img, caption=f"Preuve {i+1}")
    st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# COLONNE DROITE
st.markdown('<div class="premium-card">', unsafe_allow_html=True)
st.markdown('''
<div class="section-header">
  <svg class="section-icon" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="1.5">
    <path d="M12 14l-8-8V2h12l-4 10h6v8H8v-6h4z"/>
    <path d="M12 18v4"/>
    <path d="M8 14h8"/>
  </svg>
  <h2 class="section-title">Verdict Juridique</h2>
</div>
''', unsafe_allow_html=True)

query = st.text_input("Mots-clés juridiques", value="livraison", placeholder="Ex: livraison, remboursement, vice caché...")

# Bouton de génération du verdict - CORRIGÉ
verdict_button = st.button("⚖️ Générer le verdict", type="primary", use_container_width=True)

if verdict_button:
    if st.session_state.model is None or st.session_state.processor is None:
        st.error("⚠️ Veuillez charger le modèle d'abord")
    elif not st.session_state.messages:
        st.error("⚠️ Aucune discussion à analyser")
    else:
        with st.spinner("🧠 Analyse en cours..."):
            try:
                discussion_text = "\n".join([f"{m['auteur']}: {m['message']}" for m in st.session_state.messages])
                rag_results = rag_search(st.session_state.rag_tree, query)
                images = _maybe_shrink_images(st.session_state.uploaded_images)
                messages = build_messages(discussion_text, rag_results, images)
                inputs = _prepare_inputs(st.session_state.processor, st.session_state.model, messages)

                with torch.no_grad():
                    generated_ids = st.session_state.model.generate(
                        **inputs,
                        max_new_tokens=MAX_NEW_TOKENS,
                        do_sample=True,
                        temperature=0.3,
                        top_p=0.9,
                        pad_token_id=st.session_state.processor.tokenizer.eos_token_id,
                    )

                generated_ids_trimmed = [
                    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
                ]

                output_text = st.session_state.processor.batch_decode(
                    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
                )[0]

                st.markdown(f'''
                <div class="verdict-container">
                  <div class="verdict-header">
                    <svg width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="var(--primary)" stroke-width="1.5">
                      <path d="M12 14l-8-8V2h12l-4 10h6v8H8v-6h4z"/>
                      <path d="M12 18v4"/>
                      <path d="M8 14h8"/>
                    </svg>
                    <h3 style="margin: 0; color: var(--neutral-900);">Décision du Tribunal</h3>
                  </div>
                  <div class="verdict-text">{output_text}</div>
                </div>
                ''', unsafe_allow_html=True)
            except Exception as e:
                st.error(f"❌ Erreur: {str(e)}")

st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

In [ ]:
# Lance Streamlit en tâche de fond et l’affiche en iframe via le proxy Colab
import os, subprocess, time, signal, requests
from google.colab import output
from IPython.display import HTML, display

assert os.path.exists("app.py"), "app.py est introuvable. Ré-exécute la cellule 2."

# Tuer un éventuel process Streamlit précédent
try:
    with open('/tmp/streamlit_pid','r') as f:
        old = int(f.read().strip())
    os.killpg(os.getpgid(old), signal.SIGTERM)
    time.sleep(2)  # Attendre l'arrêt complet
except Exception:
    pass

port = 8501
env = os.environ.copy()
env["STREAMLIT_SERVER_HEADLESS"] = "true"
env["STREAMLIT_SERVER_FILE_WATCHER_TYPE"] = "none"
env["STREAMLIT_BROWSER_GATHER_USAGE_STATS"] = "false"
env["STREAMLIT_SERVER_ENABLE_CORS"] = "false"
env["STREAMLIT_SERVER_ENABLE_XSRF_PROTECTION"] = "false"

cmd = [
    "streamlit", "run", "app.py",
    "--server.port", str(port),
    "--server.headless", "true",
    "--server.fileWatcherType", "none",
    "--browser.gatherUsageStats", "false",
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false",
    "--global.developmentMode", "false",
    "--logger.level", "error"
]

proc = subprocess.Popen(
    cmd,
    preexec_fn=os.setsid,
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

with open('/tmp/streamlit_pid','w') as f:
    f.write(str(proc.pid))

# Health-check amélioré
ok = False
print("⏳ Démarrage de Streamlit... (cela peut prendre 20-30 secondes)")
for i in range(120):  # Augmenté à 120 tentatives (60 secondes max)
    try:
        r = requests.get(f"http://localhost:{port}/_stcore/health", timeout=1)
        if r.status_code == 200 and r.text == "ok":
            ok = True
            break
    except Exception as e:
        pass

    # Vérifier si le process est toujours vivant
    if proc.poll() is not None:
        stdout, stderr = proc.communicate()
        print(f"❌ Streamlit s'est arrêté avec le code {proc.returncode}")
        if stderr:
            print(f"Erreur: {stderr.decode()}")
        break

    time.sleep(0.5)
    if i % 10 == 0:  # Afficher un message toutes les 5 secondes
        print(f"⏳ En attente... ({i//2} secondes)")

if ok:
    print("✅ Streamlit est prêt !")
    time.sleep(1)  # Petit délai supplémentaire pour stabilité
    public_url = output.eval_js(f"google.colab.kernel.proxyPort({port}, {{'cache': false}})")
    display(HTML(f'<iframe src="{public_url}" style="width:100%; height:780px; border:0;"></iframe>'))
    print("🚀 Application lancée avec succès.")
else:
    print("❌ Le serveur Streamlit n’a pas répondu correctement.")
    print("Vérifiez app.py puis relancez cette cellule.")
    # Afficher les logs d'erreur si disponibles
    try:
        stdout, stderr = proc.communicate(timeout=2)
        if stderr:
            print(f"Logs d'erreur: {stderr.decode()}")
    except:
        pass